In [24]:
# part 1
def process_flow(part,flow):
    i = 1
    while True: # still in the flow
        # print(flow[i])
        if len(flow[i]) > 1 and flow[i][1] in '<>':
            condition = flow[i].split(':')
            stat = 'xmas'.index(flow[i][0])
            if eval(part[stat] + condition[0][1:]):
                return condition[1]
            else:
                i += 1
        else:
            return flow[i]

# main
# input workflows
lines = open("input_19.txt").read().splitlines()
workflows = []
i = 0
while lines[i] != '':
    # print(f'i = {i} workflow = {lines[i]}')
    # read the workflow
    name, leftover = lines[i].split('{')
    leftover = leftover[0:-1]
    instrs = leftover.split(',')
    instrs.insert(0,name)
    workflows.append(instrs)
    i += 1    
# print(workflows)

# make a list of names of the workflows
workflow_names = [x[0] for x in workflows]
# print(workflow_names)

total = 0
i += 1
for i in range(i,len(lines)):
    # print(f'i = {i} part = {lines[i]}')
    line = lines[i][1:-1]
    statstr = line.split(',')
    part = [0,0,0,0]
    for k in range(4):
        part[k] = statstr[k][2:]

    # print(part)
    # process the part
    next_name = 'in'
    while next_name not in 'AR':
        workflow_num = workflow_names.index(next_name)
        # print(workflow_num)
        next_name = process_flow(part,workflows[workflow_num])
        # print(next_name)
    
    if next_name == 'A':
        total += sum([int(x) for x in part])

print('sum of ratings =',total)




sum of ratings = 495298


In [2]:
# part 2
def process_flow_ranges(parts,flow_name):
    total_combos = 0
    # print(parts)
    if flow_name == 'A':  # accepted
        combos = 1
        for i in range(4):
            combos *= parts[i][1]-parts[i][0]+1
        total_combos += combos
        # print(parts,'--> Accepted combos = ',combos)
        return total_combos
    elif flow_name == 'R':  # rejected
        # print(parts,'--> Rejected')
        return total_combos
    else:  # send to another flow
        outflow_idx = workflow_names.index(flow_name)
        flow = workflows[outflow_idx]
        # print(flow)

    i = 1
    still_in_flow = True
    while still_in_flow: # still in the flow
        if len(flow[i]) > 1 and flow[i][1] in '<>':
            condition, outflow_name = flow[i].split(':')
            catidx = 'xmas'.index(condition[0])
            oldrange = parts[catidx]
            ineq = condition[1]
            checkvalue = int(condition[2:])            
            if ineq == '<':
                limits = [checkvalue-1, checkvalue]
                if oldrange[0] < limits[0]:  # true condition, divert it
                    splitparts = parts[:]
                    splitparts[catidx] = (oldrange[0],min(oldrange[1],limits[0]))
                    total_combos += process_flow_ranges(splitparts,outflow_name)
            
                if oldrange[1] > limits[1]:  # fail, keep going on this flow
                    parts[catidx] = (max(limits[1], oldrange[0]), oldrange[1])
                    i += 1
                else:
                    return total_combos
        
            else: # ineq == '>':
                limits = [checkvalue, checkvalue+1]
                if oldrange[1] > limits[1]:  # true condition, divert it
                    splitparts = parts[:]
                    splitparts[catidx] = (max(limits[1], oldrange[0]), oldrange[1])
                    total_combos += process_flow_ranges(splitparts,outflow_name)

                if oldrange[0] < limits[0]:  # fail, keep going on this flow
                    parts[catidx] = (oldrange[0],min(oldrange[1],limits[0]))
                    i += 1
                else:
                    return total_combos

        else:
            # send a new flow
            total_combos += process_flow_ranges(parts,flow[i])
            still_in_flow = False
    return total_combos

# main
# input workflows
lines = open("input_19.txt").read().splitlines()
workflows = []
i = 0
while lines[i] != '':
    # print(f'i = {i} workflow = {lines[i]}')
    # read the workflow
    name, leftover = lines[i].split('{')
    leftover = leftover[0:-1]
    instrs = leftover.split(',')
    instrs.insert(0,name)
    workflows.append(instrs)
    i += 1    
# print(workflows)

# make a list of names of the workflows
workflow_names = [x[0] for x in workflows]

# part_ranges = []  # list of part_ranges: [(1,200), (1,200), (1,200), (1,200)]
parts = [(1,4000)]*4
print(parts)

total_combos = process_flow_ranges(parts,'in')
print('total combos =',total_combos)






[(1, 4000), (1, 4000), (1, 4000), (1, 4000)]
total combos = 132186256794011


In [28]:
15320205000000 + 14486526000000 + 20576430000000 + 8281393428000 \
+ 35328000000000 + 26599296000000 + 16792704000000 + 21856640000000 \
+ 8167885440000
workflows

[['px', 'a<2006:qkq', 'm>2090:A', 'rfg'],
 ['pv', 'a>1716:R', 'A'],
 ['lnx', 'm>1548:A', 'A'],
 ['rfg', 's<537:gd', 'x>2440:R', 'A'],
 ['qs', 's>3448:A', 'lnx'],
 ['qkq', 'x<1416:A', 'crn'],
 ['crn', 'x>2662:A', 'R'],
 ['in', 's<1351:px', 'qqz'],
 ['qqz', 's>2770:qs', 'm<1801:hdj', 'R'],
 ['gd', 'a>3333:R', 'R'],
 ['hdj', 'm>838:A', 'pv']]

In [34]:
#-----  UNUSED

def find_rules(workflows):
    rules = []
    for w,flow in enumerate(workflows):
        i = 1
        # print(flow[i])
        if len(flow[i]) > 1 and flow[i][1] in '<>':
            condition = flow[i].split(':')
            # print(condition, condition[0][0], condition[0][2:])
            rules.append([condition[0][0],condition[0][1],int(condition[0][2:])])
        # print(rules)
        i += 1
    return rules

def build_part_ranges(part_ranges:list[list[tuple[int]]],rules):
    for rule in rules:
        print('rule =',rule)
        catidx = 'xmas'.index(rule[0])
        ineq = rule[1]
        if ineq == '<':
            limits = [rule[2]-1, rule[2]]
        else:
            limits = [rule[2], rule[2]+1]
        i = 0
        while i < len(part_ranges):
            popranges = part_ranges.pop(i)
            oldrange = popranges[catidx]
            if oldrange[0] < limits[0]:
                popranges[catidx] = (oldrange[0],limits[0])
                # print('popranges =',popranges)
                part_ranges.insert(0,popranges[:])
                i += 1
                
            if oldrange[1] > limits[1]:
                popranges[catidx] = (limits[1], oldrange[1])
                # print('popranges =',popranges)
                part_ranges.insert(0,popranges[:])
                i += 1
    print('part ranges:')
    [print(x) for x in part_ranges]
    return part_ranges



256000000000000